In [1]:
f_rec_log_train = open("track1/rec_log_train_sorted.txt", "r")
m = countlines(f_rec_log_train)

73209277

In [55]:
seekstart(f_rec_log_train)
m = 10000000
Ω = Array{Any, 1}()
for i = 1:m
    x = split(readline(f_rec_log_train), '\t')
    x = map(t->parse(Int32, t), x[1:4])
    push!(Ω, x)
end
println(size(Ω, 1))

10000000


In [56]:
T = Array{Any, 1}()
i = 1
while i ≤ m
    j = i
    user_id = Ω[i][1]
    while j ≤ m && Ω[j][1] == user_id
        j += 1
    end
    lst = -1
    for k = i : j - 2
        t = Ω[k + 1][4] - Ω[k][4]
        (t != 0) ? (lst = t) : (t = lst)
        push!(T, t)
    end
    (i ÷ 1000000 != j ÷ 1000000) && (println("$i"))
    global i = j
end
sort!(T)

time_stamp = Dict{Int32, Int32}()
time_stamp_true = Dict{Int32, Int32}()
block = size(T, 1) ÷ 16 + 1
i = 1
while i ≤ m
    j = i
    user_id = Ω[i][1]
    while j ≤ m && Ω[j][1] == user_id
        j += 1
    end
    lst = -1
    for k = i : j - 2
        t = Ω[k + 1][4] - Ω[k][4]
        (t != 0) ? (lst = t) : (t = lst)
        (t != -1) ? (id = searchsortedfirst(T, t) ÷ block + 1) : (id = -1)
        (haskey(time_stamp, id)) ? (time_stamp[id] += 1) : (time_stamp[id] = 1)
        (Ω[k + 1][3] == 1) && ((haskey(time_stamp_true, id)) ? (time_stamp_true[id] += 1) : (time_stamp_true[id] = 1))
    end
    (i ÷ 1000000 != j ÷ 1000000) && (println("$i"))
    global i = j
end

P = Dict{Int32, Float32}()
for x in keys(time_stamp)
    (haskey(time_stamp_true, x)) ? (P[x] = time_stamp_true[x] / time_stamp[x]) : (P[x] = 0)
end
println(time_stamp)
println(time_stamp_true)
println(P)

999984
1999959
2999948
3999677
4999924
5999988
6999996
7999971
8999966
9999955
999984
1999959
2999948
3999677
4999924
5999988
6999996
7999971
8999966
9999955
Dict(2=>1053836,-1=>419533,16=>611921,11=>601108,7=>647016,9=>600943,10=>615994,8=>572764,6=>532889,4=>606503,5=>387589,13=>613323,14=>609110,15=>611720,12=>604087,1=>702660)
Dict(2=>42031,-1=>77674,16=>93551,11=>33739,7=>51173,9=>38344,10=>34145,8=>42873,6=>43781,4=>36623,5=>31716,14=>32570,13=>34287,15=>31783,12=>34065,1=>21532)
Dict(2=>0.0398838,-1=>0.185144,16=>0.152881,11=>0.056128,7=>0.0790908,9=>0.0638064,10=>0.0554307,8=>0.0748528,6=>0.0821578,4=>0.0603839,5=>0.0818289,13=>0.0559037,14=>0.0534715,15=>0.0519568,12=>0.0563909,1=>0.0306436)


In [57]:
function read_from_log_test(f)
    x = split(readline(f), '\t')
    x = map(t->parse(Int32, t), x[1:4])
    return x
end

function test(f_rec_log_test, fout, key)
    m_test = countlines(f_rec_log_test)
    seekstart(f_rec_log_test)
    i = 1
    cur = read_from_log_test(f_rec_log_test)
    # m_test = 800
    while i <= m_test
        j = i + 1
        _Ω = Array{Any, 1}()
        push!(_Ω, cur)
        while j <= m_test
            cur = read_from_log_test(f_rec_log_test)
            (cur[1] != _Ω[1][1]) && break
            j += 1
            push!(_Ω, cur)
        end
        (i ÷ 5000000 != j ÷ 5000000) && println(i, ' ', j)
        dict = Array{Any, 1}()
        for k = 1:j-i
            x = _Ω[k]
            (k == 1) ? (t = -1) : (t = _Ω[k][4] - _Ω[k - 1][4])
            score = P[searchsortedfirst(T, t) ÷ block + 1]
            push!(dict, (score, x[2]))
        end
        sort!(dict)
        write(fout, "$(_Ω[1][1]),")
        for k = 1:min(size(dict, 1), 3)
            t = size(dict, 1) - k + 1
            write(fout, ((k == 1) ? "" : " ") * "$(dict[t][2])")
        end
        write(fout, "\n")
        i = j
    end
end

fout = open("test.csv", "w")
write(fout, "id,clicks\n")

test(open("track1/rec_log_test_pub.txt", "r"), fout, "Public")
test(open("track1/rec_log_test_pri.txt", "r"), fout, "Private")

close(fout)

4999977 5000050
9999990 10000050
14999999 15000020
4999988 5000005
9999954 10000009
14999996 15000019
